# LSTM Model to detect Heel Strike and Toe-off events

In this notebook, we will predict whether each sample is a heel strike or toe-off based on the accelerometer data. To do so, we train an LSTM (Long-Short Term Memory) model on 70% of the data and test it on the remaining 30%

The LSTM architecture has
- a lookback window of 3 timesteps
- 6 features (X,Y,Z acceleration of left foot and right foot) 
- 44 hidden nodes

**The code is the same as that in Model1.pynb, except that we use a for loop to iterate over all possible combinations of SUBJECT_ID, ACTIVITY and EVENT**

The subsequent steps to tune and test the model are as follows:

1. Subset the data based on subject, activity, and event
2. Remove noise from the data
3. Re-scale each of the features to range (0,5)
4. Split the data into training and testing sets  
5. Since we are using an LSTM architecture, we will implement a look-back function that will introduce historical data determined by the window size
6. Build and compile the Neural Network
7. Fit the model
8. Save the predictions and actual value to evaluate the model. The model is evaluated in terms of 
    - F1 score
    - Percentage of true positives
    - Mean of the absolute difference in time between true positives and corresponding GT events

In [1]:
import numpy
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras_tqdm import TQDMNotebookCallback
from keras.callbacks import EarlyStopping

from matplotlib import pyplot as plt

/Users/tankh/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Note: before you run the code

You can adjust the list of subject Ids, activities and events in the next cell.
Some parts are specific to indoors/outdoors, so you will need to comment the part if needed, and uncomment it it's if not needed.

## 1. Specify the list of subjects, activities and events


In [2]:
####### Select Subject Id #######
# Key in a number from 1 to 20. 

# indoors
SUBJECT_ID_LIST = ['2']
# SUBJECT_ID_LIST = ['1','2','3','4','5','6','7','8','9','10','11']
# # outdoors
# SUBJECT_ID_LIST = ['12','13','14','15','16','17','18','19','20']

###### Select Activity ######
# Key in one of the options below:

# indoors
ACTIVITY_LIST = ['indoor_walk','indoor_run','indoor_walknrun']
# ACTIVITY_LIST = ['treadmill_walk','treadmill_walknrun','treadmill_run','treadmill_all','indoor_walk','indoor_run','indoor_walknrun']
# # outdoors
# ACTIVITY_LIST = ['outdoor_walk','outdoor_run','outdoor_walknrun']

###### Select Event ######
EVENT_LIST = ['LF_HS','LF_TO','RF_HS','RF_TO']

## 2. Define helper functions

We will define two functions to be used in the for loop later, one to create a indicator variable for 'treadmill_all', the other to create the dataset for feeding into the RNN.

Since there is no indicator variable for treadmill_all for subjects 1-11, let's create another column which for this. The value of this data is 1 if either treadmill_walknrun is 1 or treadmill_slope_walk is 1

In [3]:
###############################################
######## Specific to indoors only #############
###############################################

# for SUBJECT_ID 1 to 11, uncomment if otherwise
# we will use this function in the for loop later

def label_treadmill_all (row):
   if row['treadmill_walknrun'] == 1:
      return 1
   if row['treadmill_slope_walk'] == 1:
      return 1
   return 0

def label_treadmill_run (row):
   if row['treadmill_walknrun'] == 1 and row['treadmill_walk'] == 0:
      return 1
   return 0

def label_indoor_run (row):
   if row['indoor_walknrun'] == 1 and row['indoor_walk'] == 0:
      return 1
   return 0

###############################################
######## Specific to outdoors only ############
###############################################

# for SUBJECT_ID 12 to 20, uncomment if otherwise
# we will use this function in the for loop later

def label_outdoor_run (row):
   if row['outdoor_walknrun'] == 1 and row['outdoor_walk'] == 0:
      return 1
   return 0

In [4]:
# convert an array of values into a dataset matrix
# we will use this function in the for loop later
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0:6]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 6])
	return numpy.array(dataX), numpy.array(dataY)

## 3. Train Model and save Predictions

If training for indoor activity, use the next cell and comment the cell after that.

If training for indoor activity, comment the next cell and use the cell after that.

In [5]:
for SUBJECT_ID in SUBJECT_ID_LIST:
    for ACTIVITY in ACTIVITY_LIST:
        for EVENT in EVENT_LIST:
            # read in the data
            DATA_PATH = './Combined Data_csv format/'
            df = pd.read_csv(DATA_PATH + 'Sub_'+ SUBJECT_ID + '.csv', header = 0)          
            df = df.drop(df.columns[0], axis=1)
            df['treadmill_all']=df.apply (lambda row: label_treadmill_all(row),axis=1)
            df['treadmill_run']=df.apply (lambda row: label_treadmill_run(row),axis=1)
            df['indoor_run']=df.apply (lambda row: label_indoor_run(row),axis=1)
            
            print("Currently running for: " + " -Subject: " + SUBJECT_ID + ' -Activity: ' + ACTIVITY + ' -Event: ' + EVENT)

            # Subset out the data by activity of interest
            k1=df[df[ACTIVITY]==1]
            k2 = k1[['accX_LF','accY_LF','accZ_LF','accX_RF','accY_RF','accZ_RF', EVENT]]
            
            # Remove noise from data
            k2['accX_LF_median']=k2['accX_LF'].rolling(window=3).mean()
            k2['accY_LF_median']=k2['accY_LF'].rolling(window=3).mean()
            k2['accZ_LF_median']=k2['accZ_LF'].rolling(window=3).mean()
            k2['accX_RF_median']=k2['accX_RF'].rolling(window=3).mean()
            k2['accY_RF_median']=k2['accY_RF'].rolling(window=3).mean()
            k2['accZ_RF_median']=k2['accZ_RF'].rolling(window=3).mean()

            k3 = k2[['accX_LF_median','accY_LF_median','accZ_LF_median','accX_RF_median','accY_RF_median','accZ_RF_median', EVENT]]
            k3 = k3.iloc[2:]
            dataset = k3.values

            # normalize the dataset
            scaler = MinMaxScaler(feature_range=(0, 5))
            dataset = scaler.fit_transform(dataset)

            # split into train and test sets
            train_size = int(len(dataset) * 0.7)
            test_size = len(dataset) - train_size
            train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

            look_back=3 # 3 timesteps
            trainX, trainY = create_dataset(train, look_back)
            testX, testY = create_dataset(test, look_back)

            model = Sequential()

            # Recurrent layer
            model.add(LSTM(44, return_sequences=False, dropout=0.1, recurrent_dropout=0.1))
            # Fully connected layer
            model.add(Dense(44, activation='relu'))
            # Dropout for regularization
            model.add(Dropout(0.5))
            # Output layer
            model.add(Dense(1, activation='sigmoid'))
            # Compile the model
            model.compile(
                optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

            # Fit the model
            es= EarlyStopping(monitor='val_loss', min_delta=0,patience=5,verbose=0, mode='auto')
            history = model.fit(trainX, trainY, validation_split = 0.33, epochs=50, batch_size=20, verbose=0, callbacks=[TQDMNotebookCallback(),es])

            # List all data in history
            print(history.history.keys())

            # summarize history for accuracy
            plt.plot(history.history['acc'])
            plt.plot(history.history['val_acc'])
            plt.title('model accuracy')
            plt.ylabel('accuracy')
            plt.xlabel('epoch')
            plt.legend(['train','validation'])
            plt.show()

            # summarize history for loss
            plt.plot(history.history['loss'])
            plt.plot(history.history['val_loss'])
            plt.title('model loss')
            plt.ylabel('loss')
            plt.xlabel('epoch')
            plt.legend(['train','validation'])
            plt.show()

            model.summary()

            testY_predict = model.predict(testX)

            predicted = []
            actual = []
            for i in range(len(testY)):
                #print(int(testY_predict[i]>0.5), int(testY[i]>0.5))
                predicted.append(int(testY_predict[i]>0.5))
                actual.append(int(testY[i]>0.5))

            PREDICTIONS_DF = pd.DataFrame(list(zip(predicted, actual)), columns=['predicted','actual'])

            # write dataframe to .csv
            SAVE_PATH = './Predicted Data_model_csv format/'
            PREDICTIONS_DF.to_csv(SAVE_PATH+'Sub'+SUBJECT_ID+'_'+ ACTIVITY + '_'+ EVENT + '.csv', encoding='utf-8')


In [6]:
# for SUBJECT_ID in SUBJECT_ID_LIST:
#     for ACTIVITY in ACTIVITY_LIST:
#         for EVENT in EVENT_LIST:
#             # read in the data
#             DATA_PATH = './Combined Data_csv format/'
#             df = pd.read_csv(DATA_PATH + 'Sub_'+ SUBJECT_ID + '.csv', header = 0)          
#             df = df.drop(df.columns[0], axis=1)
#             df['outdoor_run']=df.apply (lambda row: label_outdoor_run(row),axis=1)
            
#             print("Currently running for: " + " -Subject: " + SUBJECT_ID + ' -Activity: ' + ACTIVITY + ' -Event: ' + EVENT)

#             # Subset out the data by activity of interest
#             k1=df[df[ACTIVITY]==1]
#             k2 = k1[['accX_LF','accY_LF','accZ_LF','accX_RF','accY_RF','accZ_RF', EVENT]]
            
#             # Remove noise from data
#             k2['accX_LF_median']=k2['accX_LF'].rolling(window=3).mean()
#             k2['accY_LF_median']=k2['accY_LF'].rolling(window=3).mean()
#             k2['accZ_LF_median']=k2['accZ_LF'].rolling(window=3).mean()
#             k2['accX_RF_median']=k2['accX_RF'].rolling(window=3).mean()
#             k2['accY_RF_median']=k2['accY_RF'].rolling(window=3).mean()
#             k2['accZ_RF_median']=k2['accZ_RF'].rolling(window=3).mean()

#             k3 = k2[['accX_LF_median','accY_LF_median','accZ_LF_median','accX_RF_median','accY_RF_median','accZ_RF_median', EVENT]]
#             k3 = k3.iloc[2:]
#             dataset = k3.values

#             # normalize the dataset
#             scaler = MinMaxScaler(feature_range=(0, 5))
#             dataset = scaler.fit_transform(dataset)

#             # split into train and test sets
#             train_size = int(len(dataset) * 0.7)
#             test_size = len(dataset) - train_size
#             train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

#             look_back=3 # 3 timesteps
#             trainX, trainY = create_dataset(train, look_back)
#             testX, testY = create_dataset(test, look_back)

#             model = Sequential()

#             # Recurrent layer
#             model.add(LSTM(44, return_sequences=False, dropout=0.1, recurrent_dropout=0.1))
#             # Fully connected layer
#             model.add(Dense(44, activation='relu'))
#             # Dropout for regularization
#             model.add(Dropout(0.5))
#             # Output layer
#             model.add(Dense(1, activation='sigmoid'))
#             # Compile the model
#             model.compile(
#                 optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#             # Fit the model
#             es= EarlyStopping(monitor='val_loss', min_delta=0,patience=5,verbose=0, mode='auto')
#             history = model.fit(trainX, trainY, validation_split = 0.33, epochs=50, batch_size=20, verbose=0, callbacks=[TQDMNotebookCallback(),es])

#             # List all data in history
#             print(history.history.keys())

#             # summarize history for accuracy
#             plt.plot(history.history['acc'])
#             plt.plot(history.history['val_acc'])
#             plt.title('model accuracy')
#             plt.ylabel('accuracy')
#             plt.xlabel('epoch')
#             plt.legend(['train','validation'])
#             plt.show()

#             # summarize history for loss
#             plt.plot(history.history['loss'])
#             plt.plot(history.history['val_loss'])
#             plt.title('model loss')
#             plt.ylabel('loss')
#             plt.xlabel('epoch')
#             plt.legend(['train','validation'])
#             plt.show()

#             model.summary()

#             testY_predict = model.predict(testX)

#             predicted = []
#             actual = []
#             for i in range(len(testY)):
#                 #print(int(testY_predict[i]>0.5), int(testY[i]>0.5))
#                 predicted.append(int(testY_predict[i]>0.5))
#                 actual.append(int(testY[i]>0.5))

#             PREDICTIONS_DF = pd.DataFrame(list(zip(predicted, actual)), columns=['predicted','actual'])

#             # write dataframe to .csv
#             SAVE_PATH = './Predicted Data_model_csv format/'
#             PREDICTIONS_DF.to_csv(SAVE_PATH+'Sub'+SUBJECT_ID+'_'+ ACTIVITY + '_'+ EVENT + '.csv', encoding='utf-8')

Currently running for:  -Subject: 18 -Activity: outdoor_walknrun -Event: RF_HS


/Users/tankh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/tankh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/tankh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

KeyboardInterrupt: 